In [38]:
# import modules
import numpy as np
import pickle
import pandas as pd

In [39]:
# initialisation (indexing for model retrieval)
num_reps = 100
config_ids = [36,37,38,39] # configuration ids of the optimal hyperparameter models
replication_ids = {}
failed_experiment_IDs = []

for index, config_id in enumerate(config_ids):
    experimentIDs = np.arange(index*num_reps+1,(index+1)*num_reps+1) # corresponding experiment IDs 
    # removing experiments that didn't run successfully
    experimentIDs = np.delete(experimentIDs,failed_experiment_IDs)
    replication_ids[config_id] = experimentIDs

In [40]:
# load appropriate data and obtain samples

# initialisation
date = '2025-02-04'
early_stopping = 100
root_dir = r"C:\Users\vm2218\OneDrive - Imperial College London\PhD Project\seaducks\experiments\hpc_runs\04-02-2025\model_test_data"
root_dir_model = r"C:\Users\vm2218\OneDrive - Imperial College London\PhD Project\seaducks\experiments\hpc_runs\04-02-2025\fit_models"
file_name_prefix = "long_experiment_"
file_name_suffix = f"_date_{date}_early_stopping_{early_stopping}"

In [41]:
return_variables = ['lon','lat','id','time','u','v','config_id','replication_id','mu_1','mu_2','sigma_11','sigma_22','sigma_12']
testing_data = pd.DataFrame(columns=return_variables)

In [42]:
for config_id in config_ids:
    for ii in range(num_reps):
        with open(fr'{root_dir}/{file_name_prefix}{replication_ids[config_id][ii]}{file_name_suffix}_test_data.p', 'rb') as pickle_file:
                    data = pickle.load(pickle_file)
                    data_df = data[0]
                    means = data[1][0]
                    covs = data[1][1]
                    # adding info
                    data_df['config_id'] = config_id
                    data_df['replication_id'] = ii
                    data_df['mu_1'], data_df['mu_2'] = means[:,0] , means[:,1] 
                    data_df['sigma_11'], data_df['sigma_22'], data_df['sigma_12'] = covs[:,0,0], covs[:,1,1], covs[:,0,1]
                    outvars = data_df[return_variables]
                    testing_data = pd.concat([testing_data,outvars],ignore_index=True)

C:\Users\vm2218\AppData\Local\Temp\ipykernel_26340\2865588473.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  testing_data = pd.concat([testing_data,outvars],ignore_index=True)


In [43]:
# add residuals
testing_data['e_1'] = testing_data['u']-testing_data['mu_1']
testing_data['e_2'] = testing_data['v']-testing_data['mu_2']

In [44]:
# angle off-set
testing_data['angle_offset'] = np.einsum('ij,ij->i',testing_data[['u','v']].values,testing_data[['mu_1','mu_2']].values)
testing_data['angle_offset'] = np.divide(testing_data['angle_offset'], np.multiply(
    np.linalg.norm(testing_data[['u','v']].values,axis=1),np.linalg.norm(testing_data[['mu_1','mu_2']].values,axis=1)
    ))
testing_data['angle_offset'] = np.clip(testing_data['angle_offset'],-1,1)
testing_data['angle_offset'] = np.arccos(testing_data['angle_offset'])
testing_data['angle_offset']=np.rad2deg(testing_data['angle_offset'])

In [10]:
filehandler = open(f"analysis_data.p","wb")
pickle.dump(testing_data,filehandler,protocol=pickle.HIGHEST_PROTOCOL)

# Global Evaluation

## Point Metrics

In [77]:
# group by configuration id
point_analysis_data = testing_data.groupby(['config_id','replication_id'])[['lat','lon','id','time','u','v','config_id','replication_id','mu_1','mu_2','sigma_11','sigma_22','sigma_12','e_1','e_2','angle_offset']]

### Metrics for each replication

In [126]:
global_point_metrics_by_replication = pd.DataFrame(columns=['RMSE','MAE','MdAPE','MAAO'])

In [127]:
# calculate rmse over all data points for each replication
rmse_per_config = point_analysis_data.apply(lambda g: 100*np.sqrt(np.mean(g["e_1"]**2 + g["e_2"]**2)/2))
global_point_metrics_by_replication['RMSE']=rmse_per_config

In [128]:
# calculate mae over all data points for each replication
mae_per_config = point_analysis_data.apply(lambda g: 100*np.mean(np.abs(g["e_1"]) + np.abs(g["e_2"]))/2)
global_point_metrics_by_replication['MAE']=mae_per_config

In [129]:
# calculate maao over all data points for each replication
maao_per_config = point_analysis_data.apply(lambda g: np.mean(g['angle_offset']))
global_point_metrics_by_replication['MAAO']=maao_per_config

In [130]:
# calculate medAPE over all data points for each replication
mdAPE_per_config = point_analysis_data.apply(lambda g: np.median([100*np.abs(g["e_1"]/g["u"]), 100*np.abs(g["e_2"]/g["v"])]))
global_point_metrics_by_replication['MdAPE']=mdAPE_per_config

In [131]:
global_point_metrics_by_replication_data = global_point_metrics_by_replication.groupby('config_id')[['RMSE','MAE','MdAPE','MAAO']]

In [136]:
global_point_metrics_by_replication

RMSE        MAE      MdAPE       MAAO
config_id replication_id                                            
36        0               14.637541   9.852936  66.080252  40.820104
          1               14.723423   9.925735  65.559610  39.712297
          2               14.562164   9.821444  65.194768  40.082736
          3               15.107037  10.309083  66.898909  40.637668
          4               14.428255   9.725741  67.619112  41.490144
...                             ...        ...        ...        ...
39        95              14.152587   9.649509  66.233602  40.071270
          96              14.518828   9.820072  64.930363  39.732634
          97              14.789399   9.990455  64.201933  39.491352
          98              15.329971  10.304926  65.095423  40.107992
          99              14.396832   9.812459  65.761084  40.445346

[400 rows x 4 columns]

### Summary of Point Metrics

In [132]:
global_point_metrics_summaries = pd.DataFrame(columns=['RMSE mean','RMSE std', 'MAE mean', 'MAE std','MdAPE mean', 'MdAPE std','MAAO mean', 'MAAO std'])

In [133]:
point_metric_names = ['RMSE','MAE','MdAPE','MAAO']

In [134]:
for name in point_metric_names:
    mean = global_point_metrics_by_replication_data.apply(lambda g: np.mean(g[name]))
    std = global_point_metrics_by_replication_data.apply(lambda g: np.std(g[name],ddof=1))
    global_point_metrics_summaries[f'{name} mean']=mean
    global_point_metrics_summaries[f'{name} std']=std

In [135]:
global_point_metrics_summaries

,RMSE mean,RMSE std,MAE mean,MAE std,MdAPE mean,MdAPE std,MAAO mean,MAAO std
config_id,,,,,,,,
36,14.783894,0.459634,9.964375,0.303359,66.159957,0.948322,40.485154,0.708520
37,14.750128,0.460359,9.959707,0.302851,65.976940,0.941905,40.490486,0.722616
38,14.542571,0.454691,9.843939,0.300715,65.546610,0.955865,40.079249,0.715063
39,14.533736,0.456756,9.856823,0.302589,65.480731,0.920441,40.115957,0.714580
